In [117]:
import random
import numpy
from deap import algorithms, base, creator, tools
from xy_interpolation import *

In [89]:
def evalOneMax(individual):
    return numpy.sum(individual[0]),

def evalTestFreq(individual):
    try:
        s = make_shape(individual.pts,max_output_len=50)
    except ValueError:
        return 1000 # self-intersecting. very bad.
    fq, _, _ = find_eigenmodes(s, 5)
    return fitness(individual[0],np.array([5,10,15,20])),

def cxTwoPointCopy(ind1, ind2):
    """Execute a two points crossover with copy on the input individuals. The
    copy is required because the slicing in numpy returns a view of the data,
    which leads to a self overwritting in the swap operation. It prevents
    ::
    
        >>> import numpy
        >>> a = numpy.array((1,2,3,4))
        >>> b = numpy.array((5.6.7.8))
        >>> a[1:3], b[1:3] = b[1:3], a[1:3]
        >>> print(a)
        [1 6 7 4]
        >>> print(b)
        [5 6 7 8]
    """
    size = len(ind1)
    cxpoint1 = random.randint(1, size)
    cxpoint2 = random.randint(1, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else: # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    ind1[cxpoint1:cxpoint2], ind2[cxpoint1:cxpoint2] \
        = ind2[cxpoint1:cxpoint2].copy(), ind1[cxpoint1:cxpoint2].copy()
        
    return ind1, ind2

def checkIntersect(func): # TODO - come up with a way so that it just modifies the curve instead
    """
    Makes sure that the new curve isn't self-intersecting.
    If it is, it kills the individual by replacing it with a new random curve.
    """
    def decorator(func):
        print(func)
        def wrapper(*args, **kargs):
            offspring = func(*args, **kargs)
            for child in offspring:
                curve = interp(child.pts)
                if curve_intersects(curve):
                    _, child.pts = make_random_shape(len(child.pts[0]), scale = max(child.pts[0]))
            return offspring
        return wrapper
    return decorator



In [118]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,)) 
creator.create("Individual", numpy.ndarray, fitness=creator.FitnessMin)

# TODO - add the thickness as a gene
toolbox = base.Toolbox()
toolbox.register("pts", lambda: make_random_shape(4)[1]) 
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.pts)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", cxTwoPointCopy)
# TODO - set a scale for sigma
toolbox.register("mutate", lambda i:tools.mutGaussian(i, mu=0.0, sigma=1.0, indpb=0.05))
toolbox.register("select", tools.selTournament, tournsize=3)

# checking that the new curves don't intersect
# TODO - make this match
# toolbox.decorate("mate", checkIntersect)
# toolbox.decorate("mutate", checkIntersect)

def main():
    random.seed(64)
    
    pop = toolbox.population(n=300)
    
    # Numpy equality function (operators.eq) between two arrays returns the
    # equality element wise, which raises an exception in the if similar()
    # check of the hall of fame. Using a different equality function like
    # numpy.array_equal or numpy.allclose solve this issue.
    hof = tools.HallOfFame(1, similar=numpy.array_equal)
    
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    algorithms.eaSimple(pop, toolbox, cxpb=0.90, mutpb=0.2, ngen=30, stats=stats, halloffame=hof)

    return pop, stats, hof


In [119]:
if __name__ == "__main__":
    pop,stats,hof = main()
    print hof

gen	nevals	avg    	std     	min     	max    
0  	300   	1.97041	0.529057	0.492647	3.40661
1  	288   	1.49155	0.451856	0.0632258	4.52085
2  	274   	1.14806	0.525042	0.0632258	8.10957
3  	280   	0.916219	0.656451	-0.116128	8.39578
4  	270   	0.605391	0.738947	-6.7911  	8.61373
5  	281   	0.224002	0.862623	-6.7911  	2.88625
6  	275   	-0.0297403	0.994379	-6.7911  	7.13692
7  	272   	-0.349861 	1.28841 	-8.55872 	0.0632258
8  	276   	-0.803651 	1.75571 	-8.55872 	4.67307  
9  	266   	-2.0484   	2.21139 	-8.55872 	0.0632258
10 	285   	-3.69129  	2.32161 	-8.55872 	0.923104 
11 	285   	-5.53694  	2.20728 	-10.9668 	-0.116128
12 	277   	-7.40571  	1.62305 	-10.9668 	-3.29912 
13 	274   	-8.5929   	0.842241	-10.9668 	-6.7911  
14 	283   	-9.08101  	1.08118 	-15.3106 	-2.52246 
15 	283   	-9.98886  	1.1735  	-15.3106 	-8.55872 
16 	280   	-10.8291  	1.04315 	-17.5782 	-3.63239 
17 	280   	-11.2835  	1.29101 	-17.5782 	-5.38118 
18 	275   	-12.0337  	1.83296 	-17.5782 	-6.79959 
19 	271   	-13.6